## Pytorch数据并行

Pytorch有两种方法可以在多个GPU上切分模型和数据：`nn.DataParallel`和`nn.distributedataparallel`。`DataParallel`更易于使用（只需简单包装单GPU模型），但效率不高。目前主流的分布式训练代码都采用`nn.distributedataparallel`方法，简称DDP。

`DistributedDataParallel`通过多进程在多个GPUs间复制模型，在训练过程中，每个进程从磁盘加载batch数据，并将它们传递到其GPU。每一个GPU都有自己的前向过程，然后梯度在各个GPUs间进行All-Reduce。每一层的梯度不依赖于前一层，所以梯度的All-Reduce和后向过程同时计算，以进一步缓解网络瓶颈。在后向过程的最后，每个节点都得到了平均梯度，这样模型参数保持同步。

以上所述的过程便是典型的**数据并行**过程，也是最简单的分布式训练方法。我们便利用这个方法，来探索分布式训练最简单的打开方式。

首先，我们像上一节的示例那样，定义一个简单的神经网络来训练cifar数据集：

In [1]:
# 这部分代码定义在utils.py里面
import torch
from torch import nn
import torchvision
import os

# 网络结构（LeNet）
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU())
        self.out = nn.Sequential(
            nn.Linear(84, num_classes),
            nn.Softmax(dim=1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc(x)
        return self.out(x)
    

# 数据集加载（Cifar）
def get_dataset(path='./data'):
    DOWNLOAD = False
    if not(os.path.exists(path)) or not os.listdir(path):
    # not cifar dir or cifar is empyt dir
        DOWNLOAD = True
    else:
        print("Cifar dataset already exist in '{}', skip download".format(path))

    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset = torchvision.datasets.CIFAR10(
        root = path,
        train = True,
        transform = transform,
        download = DOWNLOAD
    )
    testset = torchvision.datasets.CIFAR10(
        root = path,
        train = False,
        transform = transform,
        download = DOWNLOAD
    )
    
    return trainset, testset

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


接下来是训练部分的代码。我们主要对这部分进行修改，以适配分布式训练：

In [6]:
import time

# 设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 模型
net = ConvNet().to(device)

# 数据集
trainset, testset = get_dataset()
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# 损失函数和优化器
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# 训练10个epoch
for epoch in range(10):
    t0 = time.time()
    
    loss_sum,acc_sum = 0,0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        loss = criteria(outputs, labels)
        
        loss_sum += loss.item()
        predict = torch.argmax(outputs, dim=1)
        acc_sum += torch.sum(predict == labels).item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: {}, Loss: {:.2f}, acc: {:.2f}, time cost: {:.2f}s".format(epoch, loss_sum/len(train_loader), acc_sum/len(trainset), time.time()-t0))

Cifar dataset already exist in './data', skip download
Epoch: 0, Loss: 2.12, acc: 0.32, time cost: 33.72s
Epoch: 1, Loss: 2.03, acc: 0.43, time cost: 33.75s
Epoch: 2, Loss: 2.00, acc: 0.46, time cost: 33.04s
Epoch: 3, Loss: 1.97, acc: 0.49, time cost: 23.91s
Epoch: 4, Loss: 1.94, acc: 0.52, time cost: 20.76s
Epoch: 5, Loss: 1.92, acc: 0.54, time cost: 41.05s
Epoch: 6, Loss: 1.90, acc: 0.55, time cost: 20.00s
Epoch: 7, Loss: 1.89, acc: 0.57, time cost: 24.13s
Epoch: 8, Loss: 1.88, acc: 0.58, time cost: 32.31s
Epoch: 9, Loss: 1.86, acc: 0.60, time cost: 33.40s


接下来，我们利用分布式DDP对这段训练主函数进行修改。**注意**，由于DDP启动有专门的方法，所以接下来这部分只是代码示例，并不能在JupyterNotebook中直接运行。在后续我们会通过JupyterNotebook操控命令行来启动分布式代码，代码的所有内容都包含在`main.py`里。

启动分布式训练离不开命令行参数`argparse`。因此我们先初始化一个`args`实例：

In [ ]:

import argparse

def get_args_parser():
    parser = argparse.ArgumentParser('Distributed training')
    
    #! 如果使用torch.distributed.launch，请添加'local_rank'这一项
    #! 由于torch.distributed.launch将会被弃用，改用torchrun，此时不需要添加'local_rank'这一项
    # parser.add_argument('--local_rank', type=int, default=0)
    
    # world_size这一项指定了进程数
    parser.add_argument('--world_size', type=int, default=1)
    
    # backend这一项指定了后端，可以是'nccl'或'gloo'，一般使用'nccl'
    parser.add_argument('--backend', type=str, default='nccl')

    # dist_url这一项指定了初始化进程组的地址，一般使用'env://'即可
    parser.add_argument("--dist-url", type=str, default="env://")
    
    parser.add_argument("--sync-bn", action="store_true")
    
    return parser.parse_args()      # 在main.py内部调用args = get_args_parser()

接下来，我们定义一个初始化分布式环境的函数，该函数接收`args`，然后建立好分布式训练的环境

In [ ]:
import torch.distributed as dist

def init_distributed_mode(args):
    
    # 利用os.environ来获取环境变量
    if "RANK" in os.environ and "WORLD_SIZE" in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ["WORLD_SIZE"])
        args.gpu = int(os.environ["LOCAL_RANK"])       
    # slurm_procid调度方式来获取环境变量
    elif "SLURM_PROCID" in os.environ:
        args.rank = int(os.environ["SLURM_PROCID"])
        args.gpu = args.rank % torch.cuda.device_count()
    elif hasattr(args, "rank"):
        pass
    else:
        print("Not using distributed mode")
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = "nccl"
    print(f"| distributed init (rank {args.rank}): {args.dist_url}", flush=True)
    
    # 使用torch.distributed.init_process_group来初始化进程组，这一步是初始化分布式训练的核心
    torch.distributed.init_process_group(
        backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size, rank=args.rank
    )
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    该函数阻断了所有在非主进程中的打印操作，非常好用
    """
    import builtins as __builtin__

    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop("force", False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print

初始化分布式环境后，针对训练主函数有两个非常重要的改变之处：

1. **模型**： 模型需要使用`nn.parallel.DistributedDataParallel`来包裹，使得模型适配多卡环境；

2. **数据集**：数据集（data_loader）需要先使用`torch.utils.data.distributed.DistributedSampler`定义采样器（这样每张卡上才能采得不一样的数据），然后在创建data_loader时**添加采样器，并将`shuffle`选项设置为`False`**

除此之外，还有两个不会报错，但在训练过程中会有很大作用的改变：

3. **Batch Norm同步**：可选项，这样能够使得模型在处理多张卡上面的数据时有更好的同步性能。因为如果每张卡单独在他们所负责的mini-batch上面做batch norm，很有可能会得到不如对整个batch做batch norm的效果

4. **数据间的all-reduce**：比如分类任务的准确率acc就不能像上面的`acc_sum/len(trainset)`，因为每张卡上面只处理了一部分的mini-batch，算出来的acc_sum只是这张卡上面的数值，还需要在多张卡之间通信，收集所有卡所算出的数值，再做最终的acc计算。

In [ ]:
def main():
    args = get_args_parser()
    init_distributed_mode(args)
    
    # 接下来的主函数部分就是普通的训练代码，但需要作出相应的调整：
    # 设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 模型
    net = ConvNet().to(device)
    #! ---------------------------------------------------------------
    if args.distributed:
        if args.sync_bn:
            net = torch.nn.SyncBatchNorm.convert_sync_batchnorm(net)
        net = torch.nn.parallel.DistributedDataParallel(net, device_ids=[args.gpu], output_device=args.gpu)
    #! ----------------------------------------------------------------

    # 数据集
    trainset, testset = get_dataset()
    #! ---------------------------------------------------------------
    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(trainset, num_replicas=args.world_size, rank=args.rank)
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False, sampler=train_sampler)
    #! ----------------------------------------------------------------
    else:
        train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

    # 损失函数和优化器
    criteria = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    # 训练10个epoch
    for epoch in range(10):
        #! ---------------------------------------------------------------
        if args.distributed:
            train_sampler.set_epoch(epoch)
        #! ----------------------------------------------------------------
        t0 = time.time()
        
        loss_sum,acc_sum = 0,0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criteria(outputs, labels)
            
            loss_sum += loss.item()
            predict = torch.argmax(outputs, dim=1)
            acc_sum += torch.sum(predict == labels).item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print("Epoch: {}, Loss: {:.2f}, acc: {:.2f}, time cost: {:.2f}s".format(epoch, loss_sum/len(train_loader), acc_sum/len(trainset), time.time()-t0))

所有这些设置已经写入main.py了，其中部分函数写入了`utils.py`。接下来就是启动该分布式脚本，观察运行情况。启动分布式脚本的方法主要有以下两种：

1. `torch.multiprocessing.spawn`: 该方法需要在代码中加入`mp.spawn`程序段启动分布式训练，而不是从程序外部启动。

2. `torch.distributed.launch`: 该方法是在命令行启动分布式训练代码。这种方法的优势在于，对于单机多卡、多机多卡，以及不同卡的数量、不同端口的值这些设置，能够更加方便地处理。目前这种方法也是更常用的方法。

**注**：在新版pytorch中，`python -m torch.distributed.launch`的方法将被弃用，而转为`torchrun`方法。先前使用`torch.distributed.launch`必须在命令行参数argparse中隐式地定义`local_rank`选项，但`torchrun`则不需要。我们这部分的代码便更多地采用`torchrun`的方法。

一个利用`torchrun`的最简单的例子如下：利用`--nproc_per_node=8`来指定使用8张卡来进行分布式训练。

In [13]:
!torchrun --nproc_per_node=8 main.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 4): env://
| distributed init (rank 1): env://
| distributed init (rank 2): env://
| distributed init (rank 0): env://
| distributed init (rank 5): env://
| distributed init (rank 3): env://
| distributed init (rank 7): env://
| distributed init (rank 6): env://
Cifar dataset already exist in './data', skip download
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/scatter_gather.py:9: UserWarning: is_namedtuple is deprecated, please use the python checks instead
  warnings.warn("is_namedtuple is deprecated, please use the python checks instead")
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/scatter_gather.py:9: UserWarning: is_namedtuple is deprecated, p

##### 不修改argparse的情况下，像下面这样运行代码会报错，无法识别`--local_rank`变量

In [14]:
!python -m torch.distributed.launch --nproc_per_node=8 main.py

/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:180: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
usage: Distributed training [-h] [--world_size WORLD_SIZE] [--backend BACKEND]
                            [--dist-url DIST_URL] [--sync-bn]
Distributed training: error: unrecognized arguments: --local_rank=6
usage: Distributed training

以上便是pytorch数据并行和启动分布式训练的一些基本代码方案了。接下来一节将会介绍一些涉及到分布式的api方法。